In [ ]:
import pandas as pd
import geopandas as gpd
import osmnx  as ox
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, Polygon, GeometryCollection
import numpy as np
import random
import math
from math import sin, cos, sqrt, atan2, radians
import datetime
from datetime import timedelta
import pyproj
from pyproj import Transformer
from pyproj import CRS
import folium
from folium import Map
from folium.plugins import HeatMap
from folium import plugins
from folium.plugins import HeatMapWithTime
import warnings
warnings.filterwarnings("ignore")

In [ ]:
place_name = 'Tartu, Tartu linn, Tartu maakond, Estonia'

In [ ]:
%%time
graph = ox.graph_from_place(place_name, network_type='walk')

In [ ]:
%%time
graph_proj = ox.project_graph(graph)

In [ ]:
%%time
nodes = ox.graph_to_gdfs(graph_proj, nodes=True, edges=False)

In [ ]:
graph_crs=nodes.crs

In [ ]:
crs_utm = graph_crs
crs_4326 = CRS.from_epsg(4326)
transformer_to_WGS= Transformer.from_crs(crs_utm, crs_4326, always_xy=True)
# transformer_to_UTM= Transformer.from_crs(crs_4326, crs_utm, always_xy=True)

In [ ]:
gdf_buildings = ox.geometries_from_place(place_name, tags={'building':True})

In [ ]:
gdf_amenities=ox.geometries_from_place(place_name, tags={"amenity":True, "leisure":True, 'tourism':True})

In [ ]:
gdf_hw=gdf_buildings.loc[:, ['geometry', 'name']].reset_index()
gdf_hw=gdf_hw.to_crs(graph_crs)

In [ ]:
gdf_hw['center_x']=gdf_hw['geometry'].centroid.x
gdf_hw['center_y']=gdf_hw['geometry'].centroid.y

In [ ]:
gdf_hw['nearest_node_id']=ox.distance.nearest_nodes(graph_proj, gdf_hw['center_x'], gdf_hw['center_y'])

In [ ]:
gdf_hw['node_y']=list(nodes.loc[gdf_hw['nearest_node_id']]['y'])
gdf_hw['node_x']=list(nodes.loc[gdf_hw['nearest_node_id']]['x'])

In [ ]:
gdf_hw['distance_to_node']=ox.distance.euclidean_dist_vec\
(gdf_hw['node_y'],gdf_hw['node_x'], gdf_hw['center_y'], gdf_hw['center_x'] )

In [ ]:
gdf_event=gdf_amenities.loc[:, ['geometry', 'name']].reset_index()
gdf_event=gdf_event.to_crs(graph_crs)

In [ ]:
gdf_event['center_x']=gdf_event['geometry'].centroid.x
gdf_event['center_y']=gdf_event['geometry'].centroid.y

In [ ]:
gdf_event['nearest_node_id']=ox.distance.nearest_nodes(graph_proj, gdf_event['center_x'], gdf_event['center_y'])

In [ ]:
gdf_event['node_y']=list(nodes.loc[gdf_event['nearest_node_id']]['y'])
gdf_event['node_x']=list(nodes.loc[gdf_event['nearest_node_id']]['x'])

In [ ]:
gdf_event['distance_to_node']=ox.distance.euclidean_dist_vec\
(gdf_event['node_y'],gdf_event['node_x'], gdf_event['center_y'], gdf_event['center_x'] )

In [ ]:
def get_meaningful_locations(G, gdf_hw, gdf_event): 
    while True: 
        home_id=random.randint(0, len(gdf_hw)-1)
        work_id=random.randint(0, len(gdf_hw)-1)
        home_node=nodes.loc[gdf_hw.iloc[home_id]['nearest_node_id']]
        work_node=nodes.loc[gdf_hw.iloc[work_id]['nearest_node_id']]
        distance_to_h=gdf_hw.iloc[home_id]['distance_to_node']
        distance_to_w=gdf_hw.iloc[work_id]['distance_to_node']
        final_distance = ox.distance.euclidean_dist_vec(home_node['y'], home_node['x'], work_node['y'], work_node['x'])\
                    + distance_to_h+distance_to_w
        if final_distance >=300: # meters
            regular_locations_ids=[]
            number_of_regular_locations=random.randint(3,5)
            i=0
            while i <=number_of_regular_locations: 
                regular_id=random.randint(0, len(gdf_event)-1)
                regular_node=nodes.loc[gdf_event.iloc[regular_id]['nearest_node_id']]
                if ox.distance.great_circle_vec(regular_node['y'], regular_node['x'], work_node['y'], work_node['x'], earth_radius=6371009)>=200: # meters
                    regular_locations_ids.append(regular_id)
                    i+=1                    
                else: 
                    continue
            return home_id, work_id, regular_locations_ids
        else: 
            continue

In [ ]:
n=-1
def get_regular_or_random_loc(regular_location_ids, gdf_event):
    global n
    while True: 
        random_id=[random.randint(0, len(gdf_event)-1)]
        array_of_ids=np.array([regular_location_ids, random_id], dtype='object')
        choose_reg_or_random=np.random.choice(array_of_ids,1,p=[0.6,0.4])[0]
        event_id=np.random.choice(choose_reg_or_random)
        if event_id!=n:
            event_x=gdf_event.iloc[event_id]['center_x']
            event_y=gdf_event.iloc[event_id]['center_y']
            event_node_id=gdf_event.iloc[event_id]['nearest_node_id']
            n=event_id
            return event_node_id, event_x, event_y, event_id
        else: 
            continue

In [ ]:
def get_static_points(startlon, startlat, user, data_array, time_start, time_end):
    time_start+=timedelta(minutes=1)
    startlon, startlat=transformer_to_WGS.transform(startlon, startlat)
    while time_start<time_end:
        random_minutes=random.randint(1,5)
        possible_forward_azimuth=random.randint(0,360)
        possible_distance=random.randint(0,5) #metres
        endLon,endLat,backAzimuth = (pyproj.Geod(ellps='WGS84')
            .fwd(startlon,startlat,possible_forward_azimuth,possible_distance))
        time_gps=time_start
        data_array.append([user+1, time_gps, endLon, endLat])
        time_start+=timedelta(minutes=random_minutes)
    return time_start.round(freq='S')

In [ ]:
def get_points_on_path(path, number_of_points): 
    distances = np.linspace(0, path.length, number_of_points)
    points = [path.interpolate(distance) for distance in distances]
    return points

In [ ]:
def get_chaotic_point(point_start, point_end, radius_of_buffer):
    points_intersection=point_start.buffer(radius_of_buffer).intersection(point_end.buffer(radius_of_buffer))
    path_between_points=LineString([point_start, point_end])
    final_intersection=points_intersection.intersection(path_between_points.buffer(2))
    min_x, min_y, max_x, max_y = final_intersection.bounds
    while True: 
        chaotic_point = Point([random.uniform(min_x, max_x), random.uniform(min_y, max_y)])
        if (chaotic_point.within(final_intersection)):
            return chaotic_point
        else: 
            continue

In [ ]:
def get_moving_points(G, start_node, end_node,  start_coords, end_coords, user, data_array, time_start,\
                      length_of_distance_m=10, mean_velocity_ms=1.1): 
    
    route = ox.distance.shortest_path(G, start_node, end_node, weight="length")
    route_nodes = nodes.loc[route]
    route_list=list(route_nodes.geometry.values)
    route_list.insert(0,Point(start_coords[0], start_coords[1])) 
    route_list.append(Point(end_coords[0], end_coords[1]))
    path = LineString(route_list)
    
    if path.length<=length_of_distance_m: 
        number_of_points=2
    else: 
        number_of_points=math.ceil(path.length/length_of_distance_m)
    
    points=get_points_on_path(path, number_of_points)

    for i in range(number_of_points):
        endLon, endLat = transformer_to_WGS.transform(points[i].x, points[i].y)
        if i != number_of_points-1: 
            chaotic_point=get_chaotic_point(points[i], points[i+1], radius_of_buffer=length_of_distance_m)
            distance_to_chaotic_point=LineString([points[i],chaotic_point]).length
            if distance_to_chaotic_point < 2.2: 
                time_to_chaotic_point=2
            else: 
                time_to_chaotic_point=distance_to_chaotic_point/mean_velocity_ms
            time_gps=time_start.round(freq='S')
            time_start+=timedelta(seconds=time_to_chaotic_point)
            data_array.append([user+1, time_gps, endLon, endLat])
            
            endLon, endLat = transformer_to_WGS.transform(chaotic_point.x, chaotic_point.y)
            distance_to_next_point=LineString([chaotic_point, points[i+1]]).length
            if distance_to_next_point < 2.2: 
                time_to_next_point=2
            else: 
                time_to_next_point=distance_to_next_point/mean_velocity_ms
            time_gps=time_start.round(freq='S')
            time_start+=timedelta(seconds=time_to_next_point) 
            data_array.append([user+1, time_gps, endLon, endLat])

        else:
            endLon, endLat = transformer_to_WGS.transform(points[i].x, points[i].y)
            time_gps=time_start.round(freq='S')
            data_array.append([user+1, time_gps, endLon, endLat])
    return time_start.round(freq='S')

In [ ]:

def random_plot(day_string, list_of_locations, day_of_week, number_of_events): 
    list_of_events_id=[]
    for r in range (number_of_events):
        result=get_regular_or_random_loc(user_locations[user][2], gdf_event)
        event_list=[result[0], result[1], result[2]]
        list_of_events_id.append(result[3])
        list_of_locations.append(event_list)
    
    with open(FILE_NAME, 'a') as file: 
        file.write(f"Day of week: {day_of_week}\n")
        file.write(f'Number of events: {number_of_events}\n')
        file.write(f"Event's ids: {list_of_events_id}\n")
        file.write(f'Length of list locations: {len(list_of_locations)}\n')        
    
    i=0
    
    
    if day_of_week>=6 and len(list_of_locations)>1: 
        stay_activity_time=get_static_points(list_of_locations[i][1], list_of_locations[i][2], user, gps_data_array, \
                                             time_start=another_day_start, time_end=day+timedelta(hours=random.randint(10,14)))
        with open(FILE_NAME, "a") as file: 
            file.write(f'It is weekend and the user stayed at home from {another_day_start} to {stay_activity_time}\n')
            file.write(f'Current amount of data: {len(gps_data_array)}\n')
            
    elif day_of_week<6:    
        
        stay_activity_time=get_static_points(list_of_locations[i][1], list_of_locations[i][2], user, gps_data_array, \
                                             time_start=another_day_start, time_end=day+timedelta(hours=random.randint(7,9)))
        with open(FILE_NAME, "a") as file: 
            file.write(f'It is weekday and the user stayed at from {another_day_start} till {stay_activity_time}\n')
            file.write(f'Current amount of data: {len(gps_data_array)}\n')

        moving_activity_time=get_moving_points(graph_proj, list_of_locations[i][0], list_of_locations[i+1][0],\
                        [list_of_locations[i][1], list_of_locations[i][2]],[list_of_locations[i+1][1], list_of_locations[i+1][2]],\
                        user, gps_data_array, time_start=stay_activity_time)
        with open(FILE_NAME, "a") as file:
            file.write(f'At {day_string} the user went to work and came there {moving_activity_time}\n')
            file.write(f'Current amount of data: {len(gps_data_array)}\n')

        stay_activity_time=get_static_points(list_of_locations[i+1][1], list_of_locations[i+1][2], user, gps_data_array, \
                                             time_start=moving_activity_time, time_end=day+timedelta(hours=random.randint(17,19)))
        with open(FILE_NAME, "a") as file:
            file.write(f'At {day_string} the user stayed at work till {stay_activity_time}\n')
            file.write(f'Current amount of data: {len(gps_data_array)}\n')
        
        i+=1
    else: 
        stay_activity_time=get_static_points(list_of_locations[i][1], list_of_locations[i][2], user, gps_data_array, \
                                             time_start=another_day_start, time_end=day+timedelta(hours=random.randint(22,26)))
        with open(FILE_NAME, "a") as file:
            file.write(f'It is weekend and the user decided to stay at home all day starting from {another_day_start} and "went to bed" at {stay_activity_time}\n')
            file.write(f'Current amount of data: {len(gps_data_array)}\n')
            file.write('-'*8+"\n\n")

        
        next_time=stay_activity_time
        return stay_activity_time
    

    while i < (len(list_of_locations)-1):
        moving_activity_time=get_moving_points(graph_proj, list_of_locations[i][0], list_of_locations[i+1][0],\
                            [list_of_locations[i][1], list_of_locations[i][2]],[list_of_locations[i+1][1], list_of_locations[i+1][2]],\
                             user, gps_data_array, time_start=stay_activity_time)
        with open(FILE_NAME, "a") as file:
            file.write(f'At {day_string} the user decided to go to an event {i} and came there {moving_activity_time}\n')
            file.write(f'Current amount of data: {len(gps_data_array)}\n')
        
        stay_activity_time=get_static_points(list_of_locations[i+1][1], list_of_locations[i+1][2], user, gps_data_array,\
                                             time_start=moving_activity_time, time_end=moving_activity_time+timedelta(hours=random.randint(1,3)))
        with open(FILE_NAME, "a") as file:
            file.write(f'At {day_string} the user stayed at event {i} till {stay_activity_time}\n')
            file.write(f'Current amount of data: {len(gps_data_array)}\n')

        i+=1
        
    if i >0:     
        moving_activity_time=get_moving_points(graph_proj, list_of_locations[i][0], list_of_locations[0][0],\
                            [list_of_locations[i][1], list_of_locations[i][2]], [list_of_locations[0][1], list_of_locations[0][2]],\
                            user, gps_data_array, time_start=stay_activity_time)
        with open(FILE_NAME, "a") as file:
            file.write(f'At {day_string} the user came to home at {moving_activity_time}\n')
            file.write(f'Current amount of data: {len(gps_data_array)}\n')
            file.write('-'*8+"\n\n")
        
        next_time=moving_activity_time
        return next_time

        

In [ ]:
number_of_users=3
date_beginning = '2022-07-22'
date_end = '2022-07-26'
date_range = pd.date_range(date_beginning, date_end, freq = 'd')

In [ ]:
user_locations={}
for user in range(number_of_users): 
    meaningful_locations=get_meaningful_locations(graph_proj, gdf_hw, gdf_event)
    user_locations[user]=[meaningful_locations[0], meaningful_locations[1], meaningful_locations[2]]

In [ ]:
%%time
FILE_NAME=r"GPS PLOTS"
gps_data_array=[]
for user in range(number_of_users):
    home_x=gdf_hw.iloc[user_locations[user][0]]['center_x']
    home_y=gdf_hw.iloc[user_locations[user][0]]['center_y']
    home_node_id=gdf_hw.iloc[user_locations[user][0]]['nearest_node']
    work_x=gdf_hw.iloc[user_locations[user][1]]['center_x']
    work_y=gdf_hw.iloc[user_locations[user][1]]['center_y']
    work_node_id=gdf_hw.iloc[user_locations[user][1]]['nearest_node']
    
    home_list=[home_node_id, home_x, home_y]
    work_list=[work_node_id, work_x, work_y]
    time_start=date_range[0]
    with open(FILE_NAME, "a") as file: 
        file.write(f'PLOT FOR USER {user+1}\n\n')
        file.write(f'HOME_ID: {user_locations[user][0]}\n')
        file.write(f'WORK_ID: {user_locations[user][1]}\n')
        file.write(f'REGULAR_IDs: {user_locations[user][2]}\n\n')
    for i in range(len(date_range)):
        another_day_start=time_start
        day=date_range[i]
        day_string=date_range[i].strftime('%Y-%m-%d %H:%M:%S')[:10]
        day_of_week=day.isoweekday()
        if day_of_week<6: 
            number_of_events=np.random.choice([0,1,2,3], 1, p=[0.6, 0.25, 0.1, 0.05])[0]
            list_of_locations=[home_list, work_list]
        else: 
            number_of_events=np.random.choice([0,1,2,3,4], 1, p=[0.1, 0.2, 0.30, 0.25, 0.15])[0]
            list_of_locations=[home_list]
        time_start=random_plot(day_string, list_of_locations, day_of_week, number_of_events)

In [ ]:
gps_data = pd.DataFrame(gps_data_array, columns = [ 'user', 'timestamp', 'lon', 'lat']).sort_values(by=['user','timestamp'])
print(f'Final amount of record: {len(gps_data)}')

user=2

df_1=gps_data[gps_data['user']==user][['timestamp','lon','lat']]

df_1['timestamp']=df_1['timestamp'].astype(str)

df_1['timestamp']=df_1['timestamp'].apply(lambda x: x[:10])

df_1['true_date']=pd.to_datetime(df_1['timestamp'], errors='coerce', format="%Y-%m-%d")

df_1=df_1.sort_values(by='true_date')

lat_long_list = []
date_strings=[]
for i in df_1['timestamp'].unique():
    date_strings.append(i)
    temp=[]
    for index, instance in df_1[df_1['timestamp'] == i].iterrows():
        temp.append([instance['lat'],instance['lon']])
    lat_long_list.append(temp)

m=Map(location=[58.378025, 26.728493], zoom_start=12)
title_html = '''
             <h3 align="center" style="font-size:20px"><b>Heatmap of gps data for one user with Timestamps</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))  
HeatMapWithTime(lat_long_list,auto_play=True,index=date_strings,max_opacity=0.5).add_to(m)

home_lon, home_lat=transformer_to_WGS.transform(gdf_hw.loc[user_locations[user-1][0]]['center_x'], gdf_hw.loc[user_locations[user-1][0]]['center_y'])
work_lon, work_lat=transformer_to_WGS.transform(gdf_hw.loc[user_locations[user-1][1]]['center_x'], gdf_hw.loc[user_locations[user-1][1]]['center_y'])

event_list=[225, 1388, 56]

for event_id in event_list:
    event_lon, event_lat=transformer_to_WGS.transform(gdf_event.loc[event_id]['center_x'], gdf_event.loc[event_id]['center_y'])
    if event_id in user_locations[user-1][2]: 
        popup_text='regular_location:'+str(event_id)
    else: 
        popup_text='random_location:'+str(event_id)
    folium.Marker(
      location=[event_lat, event_lon],
      popup=popup_text
    ).add_to(m)
    
folium.Marker(
  location=[home_lat, home_lon],
  popup='home'
).add_to(m)
folium.Marker(
  location=[work_lat, work_lon],
  popup='work'
).add_to(m)
m

In [ ]:
def getDistanceFromLatLonInKm(lat1,lon1,lat2,lon2):
    R = 6371 # Radius of the earth in km
    dLat = radians(lat2-lat1)
    dLon = radians(lon2-lon1)
    rLat1 = radians(lat1)
    rLat2 = radians(lat2)
    a = sin(dLat/2) * sin(dLat/2) + cos(rLat1) * cos(rLat2) * sin(dLon/2) * sin(dLon/2) 
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    d = R * c # Distance in km
    return d

def calc_velocity(dist_km, time_start, time_end):
    """Return 0 if time_start == time_end, avoid dividing by 0"""
    return dist_km / ((time_end - time_start).total_seconds()/3600) if time_end > time_start else 0


def get_distance_and_velocity(df, user, lon, lat, timestamp): 
    df[timestamp]=pd.to_datetime(df[timestamp], format='%Y-%m-%d %H:%M:%S')
    df['lon_end'] = df[lon].shift(-1)
    df['lat_end'] = df[lat].shift(-1)
    df['timestamp_end']= df[timestamp].shift(-1)
    df_final=pd.DataFrame()
    for i in df[user].unique():
        df_transitional=pd.DataFrame()
        df_transitional=df[df[user]==i]
        df_final=pd.concat([df_final, df_transitional])
        df_final.drop(index=df_final.index[-1],axis=0,inplace=True)
        
    df_final['dist_km'] = df_final.apply(
        lambda row: getDistanceFromLatLonInKm(
        lat1=row[lat],
        lon1=row[lon],
        lat2=row['lat_end'],
        lon2=row['lon_end']
    ),
    axis=1)
    
    df_final['velocity_kmh'] = df_final.apply(
    lambda row: calc_velocity(
        dist_km=row['dist_km'],
        time_start=row[timestamp],
        time_end=row['timestamp_end']
    ),
    axis=1
)
    return df_final

In [ ]:
df_final=get_distance_and_velocity(gps_data, 'user', 'lon', 'lat', 'timestamp')

print(df_final['velocity_kmh'].min())
print(df_final['velocity_kmh'].mean())
print(df_final['velocity_kmh'].max())

In [ ]:
df_final.iloc[220:235]

In [ ]:
df_final[df_final['velocity_kmh']>5]